In [5]:
from __future__ import print_function
import os
import os.path as op


In [6]:
beginning = '''#!/bin/bash
#SBATCH -n 16
#SBATCH -N 1
#SBATCH -o check_asm1_bedcov.out
#SBATCH -e check_asm_bedcov.err
#SBATCH -p sched_mit_chisholm

module add engaging/clc-assembly-cell
module add engaging/samtools/1.1
module add engaging/bedtools/2.22.1
'''

phagelist = "./data/phages.txt"


In [13]:
from __future__ import print_function

with open(phagelist) as ih, open("final_coverage.slurm","w") as oh:
    print(beginning, file=oh)
    for p in ih:
        phage=p.strip()

        pdir = "/nobackup1/jbrown/nahant_phage_reads/"
        wdir = "final_coverage"
        cdir = "final_bp_contigs"
        fastq = op.join(pdir,"{phage}_*.fastq".format(**locals()))
        contig_assembly = op.join(pdir, cdir, "{phage}final.fasta".format(**locals()))
        outcas = op.join(pdir, wdir, "{phage}final.clc.cas".format(**locals()))
        outbam = op.join(pdir, wdir, "{phage}final.clc.bam".format(**locals()))
        rearranged = op.join(pdir, wdir, "{phage}final.clc.sorted".format(**locals()))
        genomecov = op.join(pdir, wdir, "{phage}final.clc.genomecoverage".format(**locals()))

        clc_map_cmd = "clc_mapper -o {outcas} -p fb ss 100 300 -q {fastq} -d {contig_assembly} -a local -r ignore".format(**locals())
        clc_to_sam = "clc_cas_to_sam -a {outcas} -o {outbam}".format(**locals())
        sam_sort = "samtools sort {outbam} {rearranged}".format(**locals())
        sam_index = "samtools index {rearranged}".format(**locals())
        bed_gcov = "bedtools genomecov -ibam {rearranged}.bam -d > {genomecov}".format(**locals())
        
        print(clc_map_cmd, clc_to_sam, sam_sort, sam_index, bed_gcov, sep="\n", file=oh)
        print("\n", file=oh)


In [14]:
!open final_coverage.slurm -a textedit

In [ ]:
scp final_coverage.slurm jbrown@eofe4.mit.edu:/nobackup1/jbrown/